In [27]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("cafe").getOrCreate()
cafe_df = spark.read.csv("cafe.csv", header = True, inferSchema = True)
cafe_df.createOrReplaceTempView("cafe")

cafe_df.show(30)
cafe_df.printSchema()

+---+------------------+-----------+-------+-----------+
| id|       nombre_cafe|     origen|proceso|puntaje_sca|
+---+------------------+-----------+-------+-----------+
|  1|    La Perla Negra|   Colombia| Lavado|       87.5|
|  2|     Blue Mountain|    Jamaica| Lavado|       90.2|
|  3|    Sidamo Sunrise|    Etiopía|Natural|       88.7|
|  4|     Volcán de Oro|  Guatemala|  Honey|       86.9|
|  5|        Kona Bliss|      Hawái| Lavado|       89.1|
|  6| Yirgacheffe Dream|    Etiopía| Lavado|       91.0|
|  7|    Pacamara Power|El Salvador|Natural|       87.2|
|  8|     Geisha Legend|     Panamá| Lavado|       94.5|
|  9|Sumatra Mandheling|  Indonesia|  Honey|       85.8|
| 10|   Tarrazú Clásico| Costa Rica| Lavado|       88.0|
| 11|     Huehuetenango|  Guatemala|Natural|       86.5|
| 12|     Cerrado Dulce|     Brasil|Natural|       84.7|
| 13|          Kenya AA|      Kenia| Lavado|       89.8|
| 14|     Java Paradise|  Indonesia|  Honey|       85.2|
| 15|       San Ignacio|       

In [28]:
# 1. Show all coffees with a SCA score above 90.

spark.sql("""
        SELECT *
        FROM cafe
        WHERE puntaje_sca > 90
        ORDER BY puntaje_sca DESC ;
""").show()

+---+-----------------+-------+-------+-----------+
| id|      nombre_cafe| origen|proceso|puntaje_sca|
+---+-----------------+-------+-------+-----------+
|  8|    Geisha Legend| Panamá| Lavado|       94.5|
| 27|     Gesha Spirit| Panamá|  Honey|       93.1|
| 30|   Ethiopia Queen|Etiopía|Natural|       92.3|
|  6|Yirgacheffe Dream|Etiopía| Lavado|       91.0|
| 23|       SL28 Magic|  Kenia|Natural|       90.5|
|  2|    Blue Mountain|Jamaica| Lavado|       90.2|
+---+-----------------+-------+-------+-----------+



In [30]:
# 2. List the names and origins of coffees processed with the "Natural" method.

spark.sql("""
        SELECT nombre_cafe, origen, proceso
        FROM cafe
        WHERE proceso = "Natural";
""").show()

+--------------+-----------+-------+
|   nombre_cafe|     origen|proceso|
+--------------+-----------+-------+
|Sidamo Sunrise|    Etiopía|Natural|
|Pacamara Power|El Salvador|Natural|
| Huehuetenango|  Guatemala|Natural|
| Cerrado Dulce|     Brasil|Natural|
| Santa Bárbara|   Honduras|Natural|
| Sidra Supreme|    Ecuador|Natural|
|    SL28 Magic|      Kenia|Natural|
|Bourbon Rosado|   Colombia|Natural|
|Ethiopia Queen|    Etiopía|Natural|
+--------------+-----------+-------+



In [38]:
# 3. Count how many coffees come from Colombia.

spark.sql("""
        SELECT origen, COUNT (*) AS Pais
        FROM cafe
        WHERE origen = "Colombia"
        GROUP BY origen;
""").show()


+--------+----+
|  origen|Pais|
+--------+----+
|Colombia|   3|
+--------+----+



In [45]:
# 4. Find the average SCA score for coffees from Ethiopia.

spark.sql("""
        SELECT origen, AVG(puntaje_sca) AS Promedio
        FROM cafe
        WHERE origen = "Etiopía"
        GROUP BY origen ;
""").show()


+-------+-----------------+
| origen|         Promedio|
+-------+-----------------+
|Etiopía|90.66666666666667|
+-------+-----------------+



In [46]:
# 5. Show the top 5 coffees with the highest SCA scores.

spark.sql("""
        SELECT *
        FROM cafe
        ORDER BY puntaje_sca DESC
        LIMIT 5;
""").show()

+---+-----------------+-------+-------+-----------+
| id|      nombre_cafe| origen|proceso|puntaje_sca|
+---+-----------------+-------+-------+-----------+
|  8|    Geisha Legend| Panamá| Lavado|       94.5|
| 27|     Gesha Spirit| Panamá|  Honey|       93.1|
| 30|   Ethiopia Queen|Etiopía|Natural|       92.3|
|  6|Yirgacheffe Dream|Etiopía| Lavado|       91.0|
| 23|       SL28 Magic|  Kenia|Natural|       90.5|
+---+-----------------+-------+-------+-----------+



In [47]:
# 6. List all unique processing methods in the table.

spark.sql("""
        SELECT Distinct(proceso)
        FROM cafe
""").show()



+-------+
|proceso|
+-------+
|  Honey|
| Lavado|
|Natural|
+-------+



In [49]:
# 7. Show the number of coffees per country of origin, sorted from most to least.

spark.sql("""
        SELECT origen, COUNT(*) AS Numero_de_cafes
        FROM cafe
        GROUP BY origen
        ORDER BY Numero_de_cafes DESC;
""").show()



+-----------+---------------+
|     origen|Numero_de_cafes|
+-----------+---------------+
|    Etiopía|              3|
|  Indonesia|              3|
|  Guatemala|              3|
|     Brasil|              3|
|   Colombia|              3|
|      Kenia|              2|
|     Panamá|              2|
|El Salvador|              2|
|    Ecuador|              1|
|     Ruanda|              1|
|    Bolivia|              1|
|      Hawái|              1|
|   Honduras|              1|
|    Jamaica|              1|
|       Perú|              1|
| Costa Rica|              1|
|     México|              1|
+-----------+---------------+



In [53]:
# 8. Find the coffee(s) with the lowest SCA score.

spark.sql("""
        SELECT nombre_cafe, origen, puntaje_sca
        FROM cafe
        ORDER BY puntaje_sca ASC
        LIMIT 1;
""").show()


+-------------+------+-----------+
|  nombre_cafe|origen|puntaje_sca|
+-------------+------+-----------+
|Cerrado Dulce|Brasil|       84.7|
+-------------+------+-----------+



In [60]:
# 9. List all coffees whose name contains the word "Gold" or "Oro".

spark.sql("""
        SELECT nombre_cafe, origen
        FROM cafe
        WHERE nombre_cafe LIKE '%Gold%' OR 
              nombre_cafe LIKE '%Oro%';
""").show()


+-------------+---------+
|  nombre_cafe|   origen|
+-------------+---------+
|Volcán de Oro|Guatemala|
|    Obata Oro|   Brasil|
+-------------+---------+



In [61]:
# 10. Show the average SCA score for each processing method.

spark.sql("""
        SELECT proceso, AVG (puntaje_sca) AS promedio
        FROM cafe
        WHERE
        GROUP BY proceso
        ORDER BY promedio DESC;
""").show()


+-------+-----------------+
|proceso|         promedio|
+-------+-----------------+
| Lavado|88.44999999999997|
|Natural|88.37777777777778|
|  Honey|86.97142857142856|
+-------+-----------------+

